In [1]:
# import libraries
import requests
import time
from datetime import datetime
import json
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
colors = ["#118AB2", "#EF476F", "#FFD166", "#06D6A0", "#EE754D", "#002E99"]
#plt.style.use('seaborn-whitegrid')

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
# mysql login info
sys.path.insert(0, '../../Key')
from mysql_secret import dbuser, dbpass, dbhost, dbname
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

In [3]:
def flatten_cols(df):
    df.columns = [
        '_'.join(tuple(map(str, t))).strip() 
        for t in df.columns.values
        ]
    return df

In [4]:
# Establish Twitter connection
import tweepy

# get tweepy credentials
sys.path.insert(0, '../../Key')
from tweepy_keys import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

# Upcoming Week Schedule

In [ ]:
connection = engine.connect()



In [5]:
# Post a tweet for each record
tweet_text = "Upcoming IPOs:\n"
for row in this_weeks_ipos_df.itertuples():
    
    # remove .00 from proposed share prices to reduce unecesary characters
    psp = row.proposed_share_price.replace(".00", "")
    
    tweet_text += "\n"
    row_text = f"{row.price_month_day}: ${row.symbol} proposed price ${psp}" 
    tweet_text += row_text
print(tweet_text)
print(f"******\nCharacter Count: {len(tweet_text)}")


NameError: name 'this_weeks_ipos_df' is not defined

In [ ]:
# Send out the tweet
# api.update_status(tweet_text)  

# Stock Goes Live Today

In [ ]:
# get todays date
today = date(2021, 1, 21).strftime("%Y-%m-%d")  #date for testing, not actually today
# today = datetime.today().strftime("%Y-%m-%d")
print(today)

In [ ]:
# get stocks that go live today
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
df = pd.read_sql(f"""
    SELECT * 
    FROM stocks s
    LEFT JOIN company_info ci
        ON s.symbol = ci.symbol
    WHERE priced_date = '{today}'
        AND exchange <> 'NASDAQ Capital'""", connection)
connection.close()
df

In [ ]:
tweet_text = f"${df['symbol'][0]} {df['company'][0]} should trade today"
tweet_text

# Performance Analysis

In [ ]:
connection = engine.connect()
ipo_df = pd.read_sql("SELECT * FROM ipo_tracker.vw_stocks_performance", connection)
connection.close()
ipo_df.head()

## Sector and Industry

In [ ]:
# remove stocks with None or blank as sector / industry
si_data_df = ipo_df[(ipo_df["industry"].notnull()) & (ipo_df["industry"] != "")
            & (ipo_df["sector"].notnull()) & (ipo_df["sector"] != "")]
si_data_df.head(2)

### Sector by Year

In [ ]:
sector_by_year_df = si_data_df.pivot_table(index='sector', columns='first_trade_year', 
                                    aggfunc={"symbol": 'count'})
sector_by_year_df = sector_by_year_df.fillna(0).sort_values(by=sector_by_year_df.columns[-1], ascending=True)

# pass to function to flatten columns
sector_by_year_df = flatten_cols(sector_by_year_df)
sector_by_year_df


In [ ]:
ax = sector_by_year_df.plot(kind="barh", figsize=(8,7), color=colors, zorder=3, width=0.75)
ax.grid(which="major", axis='both', color='#e6e6e6', zorder=0)
ax.legend([2018, 2019, 2020])
ax.set_ylabel(f'Sector', fontsize=12)
ax.set_xlabel(f'IPOs Count', fontsize=12)
ax.set_title("IPO Count by Sector", fontsize=20)
plt.show()

## Top Performing n Period

Values for analysis

In [ ]:
# example stock to view
stock_symbol = 'ABCM' 

Data preparation

In [ ]:
# add price change column
sa_df = ipo_df.copy()
sa_df["first_close_to_rec_close"] = sa_df["most_recent_close"] / sa_df["first_day_close"] - 1
sa_df.head(5)

In [ ]:
# get sector for stock of interest
stock_sector = sa_df.loc[sa_df["symbol"] == stock_symbol]["sector"].iloc[0]
print(f"{stock_symbol} is in the {stock_sector} sector")

# get industry for stock of interest
stock_industry = sa_df.loc[sa_df["symbol"] == stock_symbol]["industry"].iloc[0]
print(f"{stock_symbol} is in the {stock_industry} industry")

# get market cap classification for stock of interest
stock_market_cap = sa_df.loc[sa_df["symbol"] == stock_symbol]["market_cap_classification"].iloc[0]
print(f"{stock_symbol} is a {stock_market_cap} market cap")

In [ ]:
# create new columns for vizualization
# sector
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["sector"] == stock_sector)]
choices = [stock_symbol, stock_sector]
sa_df["Stock Sector"] = np.select(conditions, choices, "Other")

# industry
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_industry)]
choices = [stock_symbol, stock_industry]
sa_df["Stock Industry"] = np.select(conditions, choices, "Other")

# market cap
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_market_cap)]
choices = [stock_symbol, stock_market_cap]
sa_df["Stock Market Cap"] = np.select(conditions, choices, "Other")

sa_df.head(2)

In [ ]:
# data for plotting sector in similar market cap
sa_mc_df = sa_df[sa_df["market_cap_classification"] == stock_market_cap]
sa_mc_df

In [ ]:
# Plot
colors = ["#118AB2", "#EF476F", "#d9d9d9"]
sns.lmplot('days_trading', 'first_close_to_rec_close', data=sa_mc_df, hue='Stock Sector', 
           palette=colors[0:3], scatter_kws={"s": 100}, fit_reg=True, height=6, aspect=1.6)
plt.title("Performance Over Time", fontsize=20)
plt.suptitle(f"{stock_market_cap} IPOs", fontsize=15)
plt.ylabel("YOY Change")
plt.xlabel("Days Trading")
plt.grid(which="major", axis='both', color='#e6e6e6', zorder=0)


In [ ]:
# remove stocks with None or blank as sector / industry
ax1 = sa_df.plot(kind="scatter", x="market_cap", y="first_close_to_rec_close", c=colors[0], figsize=(8,7), zorder=3)
ax.margins(0.05)

In [ ]:
sns.relplot(
    data=sa_df,
    x="days_trading", y="first_close_to_rec_close", col="market_cap_classification", col_wrap=3,
    hue="Stock Sector", palette=colors[0:3]
)